# Passing ancillary information to evaluate()

This notebook explores passing anciallary information for evaluating individuals.

For example, each `leap_ec.distributed.individual.DistributedIndividual` supports adding a UUID for each newly created individual.  This UUID string can be used to create a file or subdirectory name that can be later associated with that individual.  For example, a deep-learner model can use that as a file name, or a file containing frames for a driving animation can be stored in a subdirectory that uses that UUID as a name.  However, `Problem.evaluate()` has no direct support for passing that UUID to support either of those scenarios, or any other similar ancillary information.

This notebook poses two solutions: one where `*args, **kwargs` is added to `Problem.evluate()` and the other where a special `Decoder` that adds that ancillary information is given.

In [9]:
import random
from math import nan

from pprint import pformat

import leap_ec

from leap_ec.core import Decoder
from leap_ec.problem import ScalarProblem
from leap_ec.distributed.individual import DistributedIndividual

## Tailoring an `Individual` and `Problem` to accept ancillary evaluation information
First we'll demonstrate extending `Individual` and tailoring a corresponding `Problem` subclass to allow for passing ancially information to `Problem.evaluate()`.

In [2]:
# Define a problem class that uses UUID during evaluation
class UUIDProblem(ScalarProblem):
    def __init__(self):
        super().__init__(maximize=True)
        
    def evaluate(self, phenome, *args, **kwargs):
        print(f'UUIDProblem.evaluate(), phenome: {str(phenome)}, uuid: {str(kwargs["uuid"])}')
        # Just return a random number because we only care about ensuring the UUID made it to evaluate
        return random.random()

In [3]:
# We need to subclass DistributedIndividual to ensure that it passes the UUID for evaluations;
# DistributedIndividual implicitly tags each newly created individual with a UUID, which is
# why we're using it for this example instead of Individual.
class MyDistributedIndividual(DistributedIndividual):
    def __init__(self,genome, decoder=None, problem=None):
        super().__init__(genome, decoder, problem) # superclass will also set self.uuid
        
    def evaluate(self):
        """ we copy the contents of core.Individual.evaluate() to capture semantics,
            and extend it to pass the UUID to evaluate()
        """
        try:
            self.fitness = self.problem.evaluate(self.decode(), uuid=self.uuid)
            self.is_viable = True # we were able to evaluate
        except Exception as e:
            self.fitness = nan
            self.exception = e
            self.is_viable = False # we could not complete an eval

        return self.fitness

In [4]:
# Now test this scheme out by creating an example individual
ind = MyDistributedIndividual([], decoder=leap_ec.core.IdentityDecoder(), problem=UUIDProblem())

fitness = ind.evaluate()

print(f'ind uuid: {ind.uuid}, ind.fitness: {ind.fitness}')

UUIDProblem.evaluate(), phenome: [], uuid: 85fa7336-a225-4dbf-8600-c0f700b416a5
ind uuid: 85fa7336-a225-4dbf-8600-c0f700b416a5, ind.fitness: 0.6586321029838671


Since we had full control over `UUIDProblem.evaluate()` we could have explicitly added a `uuid` keyword parameter, especially since our tailored `MyDistributedIndividaul.evaluate()` was going to be the only `evaluate()` to call it.  However, this did have a disadvantage in that we had to faithfully copy over the original `Individual.evaluate()` and hack to fit to ensure the exception handling semantics made it over.

## Extending a `Decoder` to contain ancillary data to pass to a tailored `Problem`

The next approach entails extending the `phenome`, itself, to contain the desired ancillary data.  That is, `Problem.evalute()` accepts a `phenome` parameter, not a `genome` as happens with other, extand EA toolkits, and we can piggyback that ancillary data in the phenome. During the evaluation an individual's genome is decoded into a phenome meaningful to a given `Problem` instance.  Of course for representations that are already phenotypic, such as real-valued vectors, the decoder can be `IdentifyDecoder` that just faithfully passes along the `genome` as the `phenome` without change.

Though there is generally a one-to-one mapping between the genome and phenome, it needn't be that way.  That is, the phenotypic space can contain information not found in the genome.  For example, alleles will dictate our hair color, but we can still dye hair a different color.  So, extending the phenome to include additional information is a more "evolutionary correct" approach to using ancillary information.

In our test implementation we modify the `Decoder` abstract base class to have `decode` accept arguments for what individual attributes, such as the UUID, we want to pass to `Problem.evaluate()`.  We once again use the familiar `*args, **kwargs` pattern.

(This is what changed in the base implementation.)
```
class Decoder(abc.ABC):
    @abc.abstractmethod
    def decode(self, genome, *args, **kwargs): # added optional args and kwargs
        pass
```

So, now to use this modified version of the decoder.

In [20]:
# Define a problem class that uses UUID during evaluation
class DifferentUUIDProblem(ScalarProblem):
    def __init__(self):
        super().__init__(maximize=True)
        
    def evaluate(self, phenome):
        print(f'DifferentUUIDProblem.evaluate(), phenome: {pformat(repr(phenome))}')
        # Just return a random number because we only care about ensuring the UUID made it to evaluate
        return random.random()

In [21]:
class ExtraIdentityDecoder(Decoder):
    def __init__(self):
        super().__init__()

    def decode(self, genome, *args, **kwargs):
        print(f'decode(): genome: {genome}, args: {args}, kwargs: {kwargs}')
        return {'phenome' : genome, 'args' : args, 'kwargs' : kwargs}

    def __repr__(self):
        return type(self).__name__ + "()"

In [22]:
class MyOtherDistributedIndividual(DistributedIndividual):
    def __init__(self,genome, decoder=None, problem=None):
        super().__init__(genome, decoder, problem) # superclass will also set self.uuid
        
    def decode(self, *args, **kwargs): # Have to extend this, too
        return self.decoder.decode(self.genome, args, kwargs)
        
    def evaluate(self):
        """ we copy the contents of core.Individual.evaluate() to capture semantics,
            and extend it to pass the UUID to evaluate()
        """
#         phenome = self.decode(self.genome, uuid=self.uuid)
        phenome = self.decode(self.genome, uuid=self.uuid)
        try:
            self.fitness = self.problem.evaluate(phenome)
            self.is_viable = True # we were able to evaluate
        except Exception as e:
            self.fitness = nan
            self.exception = e
            self.is_viable = False # we could not complete an eval

        return self.fitness

In [23]:
# Now to test it out.

ind = MyOtherDistributedIndividual([], decoder=ExtraIdentityDecoder(), problem=DifferentUUIDProblem())

fitness = ind.evaluate()

print(f'ind uuid: {ind.uuid}, ind.fitness: {ind.fitness}')

decode(): genome: [], args: (([],), {'uuid': UUID('f8c512d7-1f1f-4b97-8d97-fd25ae7c3102')}), kwargs: {}
DifferentUUIDProblem.evaluate(), phenome: ("{'phenome': [], 'args': (([],), {'uuid': "
 "UUID('f8c512d7-1f1f-4b97-8d97-fd25ae7c3102')}), 'kwargs': {}}")
ind uuid: f8c512d7-1f1f-4b97-8d97-fd25ae7c3102, ind.fitness: 0.716198188667923
